# Instalación de la Biblioteca `openpyxl`

Para trabajar con archivos de Excel en Python, una de las bibliotecas más útiles y populares es `openpyxl`. Esta biblioteca permite leer, escribir y modificar archivos Excel con extensión `.xlsx`. A continuación, te explicamos cómo instalar `openpyxl` en tu entorno de desarrollo.

In [14]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


# Importaciones necesarias

In [15]:
import pandas as pd

# Transformación del dataset de histórico de calificaciones `DATA_HIST`

Se transforma a un formato de 1 registro por ID.

In [16]:
# Cargar el dataset
data = pd.read_excel('DATA_HIST.xlsx')


# Crear las columnas de repetición para 1 ESO, 1 FPB, 2 ESO
data['REPITIENDO_1_ESO'] = data.apply(lambda row: 1 if row['CURSO'] == '1 ESO' and row['REPITIENDO'] == 'S' else 0, axis=1)
data['REPITIENDO_1_FPB'] = data.apply(lambda row: 1 if row['CURSO'] == '1 FPB' and row['REPITIENDO'] == 'S' else 0, axis=1)
data['REPITIENDO_2_ESO'] = data.apply(lambda row: 1 if row['CURSO'] == '2 ESO' and row['REPITIENDO'] == 'S' else 0, axis=1)


# Agrupar los resultados por ID_ESTUDIANTE
repeating_status = data.groupby('ID_ESTUDIANTE').agg({
    'REPITIENDO_1_ESO': 'max',
    'REPITIENDO_1_FPB': 'max',
    'REPITIENDO_2_ESO': 'max'
}).reset_index()

# Derretir el DataFrame
melted = data.melt(id_vars=['ID_ESTUDIANTE', 'CURSO', 'MATERIA', 'REPITIENDO', 'PROMOCIONA'], 
                   value_vars=['1_EVALUACION', '2_EVALUACION', 'EVALUACION_FINAL'], 
                   var_name='EVALUACION', value_name='NOTA')

# Crear la columna CURSO_MATERIA_EVALUACION
melted['CURSO_MATERIA_EVALUACION'] = melted['CURSO'] + "_" + melted['MATERIA'] + "_" + melted['EVALUACION']

# Pivotar el DataFrame
pivoted = melted.pivot_table(index='ID_ESTUDIANTE', columns='CURSO_MATERIA_EVALUACION', values='NOTA', aggfunc='first').reset_index()

# Sumar las recuperaciones por ID_ESTUDIANTE y campo de recuperación
recovery_counts = data[['ID_ESTUDIANTE', '1_RECUPERACION', '2_RECUPERACION', 'EV_EXTRAORDINARIA']]

# Convertir 'S' a 1 y 'N' a 0 para poder sumar
recovery_counts = recovery_counts.replace({'S': 1, 'N': 0})

# Sumar las recuperaciones para cada ID_ESTUDIANTE
recovery_summary = recovery_counts.groupby('ID_ESTUDIANTE').sum().add_prefix('RECUPERACIONES_')

# Extraer valores de PROMOCIONA para 1 FPB
promociona_fp1 = data[(data['CURSO'] == '1 FPB') & (data['MATERIA'] == 'Media curso')][['ID_ESTUDIANTE', 'PROMOCIONA']].drop_duplicates().set_index('ID_ESTUDIANTE')

# Combinar todo
result = pivoted.merge(repeating_status, on='ID_ESTUDIANTE', how='left').merge(recovery_summary, on='ID_ESTUDIANTE', how='left')

# Añadir PROMOCIONA para 1 FPB
result = result.merge(promociona_fp1, on='ID_ESTUDIANTE', how='left').rename(columns={'PROMOCIONA': 'PROMOCIONA_1_FPB'})

# Eliminar duplicados y quedarse con el primer registro del primer año (REPITIENDO a N) NO PROMOCIONA
result = result.sort_values(by='PROMOCIONA_1_FPB', ascending=False).drop_duplicates(subset=['ID_ESTUDIANTE'], keep='last')

# Ordenar por id
result = result.sort_values(by='ID_ESTUDIANTE')

result.head()


,ID_ESTUDIANTE,1 ESO_Castellano_1_EVALUACION,1 ESO_Castellano_2_EVALUACION,1 ESO_Castellano_EVALUACION_FINAL,1 ESO_E. Física_1_EVALUACION,1 ESO_E. Física_2_EVALUACION,1 ESO_E. Física_EVALUACION_FINAL,1 ESO_Inglés_1_EVALUACION,1 ESO_Inglés_2_EVALUACION,1 ESO_Inglés_EVALUACION_FINAL,...,2 ESO_Media curso_1_EVALUACION,2 ESO_Media curso_2_EVALUACION,2 ESO_Media curso_EVALUACION_FINAL,REPITIENDO_1_ESO,REPITIENDO_1_FPB,REPITIENDO_2_ESO,RECUPERACIONES_1_RECUPERACION,RECUPERACIONES_2_RECUPERACION,RECUPERACIONES_EV_EXTRAORDINARIA,PROMOCIONA_1_FPB
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,2.0,0.0,1.0,N
1,2,6.0,5.0,5.0,5.0,5.0,8.0,3.0,4.0,3.0,...,4.9,4.2,5.1,0,1,0,4.0,4.0,2.0,N
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0.0,0.0,1.0,S
4,4,2.0,3.0,4.0,3.0,5.0,4.0,5.0,5.0,5.0,...,4.7,4.8,4.8,1,0,0,2.0,0.0,2.0,S
5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0.0,0.0,0.0,S


# Reorganización del orden de columnas

In [17]:
#Reorganizamos orden de columnas

# Ordenar las columnas de las evaluaciones
columns_order = list(result.columns)

# Obtener las columnas específicas de evaluaciones
ID = [col for col in columns_order if 'ID' in col]
evaluaciones_1_eso = [col for col in columns_order if '1 ESO' in col]
evaluaciones_2_eso = [col for col in columns_order if '2 ESO' in col]
evaluaciones_1_fpb = [col for col in columns_order if '1 FPB' in col]
reptiendo_1= [col for col in columns_order if 'REPITIENDO_1_ESO' in col]
reptiendo_2 = [col for col in columns_order if 'REPITIENDO_2_ESO' in col]
reptiendo_FPB = [col for col in columns_order if 'REPITIENDO_1_FPB' in col]
# Obtener el resto de las columnas
resto_columnas = [col for col in columns_order if col not in evaluaciones_2_eso and col not in evaluaciones_1_fpb and col not in evaluaciones_1_eso and col not in ID and col not in reptiendo_1 and col not in reptiendo_2 and col not in reptiendo_FPB]

# Combinar las columnas en el orden deseado
new_columns_order = ID + evaluaciones_1_eso + evaluaciones_2_eso + evaluaciones_1_fpb + reptiendo_1 + reptiendo_2 + reptiendo_FPB + resto_columnas

# Reorganizar las columnas del DataFrame result
result = result[new_columns_order]

result.head()

,ID_ESTUDIANTE,1 ESO_Castellano_1_EVALUACION,1 ESO_Castellano_2_EVALUACION,1 ESO_Castellano_EVALUACION_FINAL,1 ESO_E. Física_1_EVALUACION,1 ESO_E. Física_2_EVALUACION,1 ESO_E. Física_EVALUACION_FINAL,1 ESO_Inglés_1_EVALUACION,1 ESO_Inglés_2_EVALUACION,1 ESO_Inglés_EVALUACION_FINAL,...,1 FPB_Ámbito de Comunicación y Ciencias Sociales 1º_1_EVALUACION,1 FPB_Ámbito de Comunicación y Ciencias Sociales 1º_2_EVALUACION,1 FPB_Ámbito de Comunicación y Ciencias Sociales 1º_EVALUACION_FINAL,REPITIENDO_1_ESO,REPITIENDO_2_ESO,REPITIENDO_1_FPB,RECUPERACIONES_1_RECUPERACION,RECUPERACIONES_2_RECUPERACION,RECUPERACIONES_EV_EXTRAORDINARIA,PROMOCIONA_1_FPB
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,5.0,5.0,0,0,0,2.0,0.0,1.0,N
1,2,6.0,5.0,5.0,5.0,5.0,8.0,3.0,4.0,3.0,...,5.0,5.0,5.0,0,0,1,4.0,4.0,2.0,N
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,5.0,6.0,0,0,0,0.0,0.0,1.0,S
4,4,2.0,3.0,4.0,3.0,5.0,4.0,5.0,5.0,5.0,...,5.0,5.0,6.0,1,0,0,2.0,0.0,2.0,S
5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,7.0,8.0,0,0,0,0.0,0.0,0.0,S


In [18]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37 entries, 0 to 44
Data columns (total 50 columns):
 #   Column                                                                Non-Null Count  Dtype  
---  ------                                                                --------------  -----  
 0   ID_ESTUDIANTE                                                         37 non-null     int64  
 1   1 ESO_Castellano_1_EVALUACION                                         27 non-null     float64
 2   1 ESO_Castellano_2_EVALUACION                                         27 non-null     float64
 3   1 ESO_Castellano_EVALUACION_FINAL                                     27 non-null     float64
 4   1 ESO_E. Física_1_EVALUACION                                          27 non-null     float64
 5   1 ESO_E. Física_2_EVALUACION                                          27 non-null     float64
 6   1 ESO_E. Física_EVALUACION_FINAL                                      27 non-null     float64
 7   1

# Renombramiento de las columnas

In [19]:
# Renombrar las columnas según las indicaciones
nuevos_nombres = {
    #CAMBIOS PARA 1ESO
    '1 ESO_Castellano_1_EVALUACION': '1ESO_C_1EV',
    '1 ESO_Castellano_2_EVALUACION': '1ESO_C_2EV',
    '1 ESO_Castellano_EVALUACION_FINAL': '1ESO_C_EVF',
    '1 ESO_E. Física_1_EVALUACION': '1ESO_EF_1EV',
    '1 ESO_E. Física_2_EVALUACION': '1ESO_EF_2EV',
    '1 ESO_E. Física_EVALUACION_FINAL': '1ESO_EF_EVF',
    '1 ESO_Inglés_1_EVALUACION': '1ESO_I_1EV',
    '1 ESO_Inglés_2_EVALUACION': '1ESO_I_2EV',
    '1 ESO_Inglés_EVALUACION_FINAL': '1ESO_I_EVF',
    '1 ESO_Matemáticas_1_EVALUACION': '1ESO_M_1EV',
    '1 ESO_Matemáticas_2_EVALUACION': '1ESO_M_2EV',
    '1 ESO_Matemáticas_EVALUACION_FINAL': '1ESO_M_EVF',
    '1 ESO_Media curso_1_EVALUACION': '1ESO_MEDIA_1EV',
    '1 ESO_Media curso_2_EVALUACION': '1ESO_MEDIA_2EV',
    '1 ESO_Media curso_EVALUACION_FINAL': '1ESO_MEDIA_EVF',
    # CAMBIOS PARA 2ESO
    '2 ESO_Castellano_1_EVALUACION': '2ESO_C_1EV',
    '2 ESO_Castellano_2_EVALUACION': '2ESO_C_2EV',
    '2 ESO_Castellano_EVALUACION_FINAL': '2ESO_C_EVF',
    '2 ESO_E. Física_1_EVALUACION': '2ESO_EF_1EV',
    '2 ESO_E. Física_2_EVALUACION': '2ESO_EF_2EV',
    '2 ESO_E. Física_EVALUACION_FINAL': '2ESO_EF_EVF',
    '2 ESO_Inglés_1_EVALUACION': '2ESO_I_1EV',
    '2 ESO_Inglés_2_EVALUACION': '2ESO_I_2EV',
    '2 ESO_Inglés_EVALUACION_FINAL': '2ESO_I_EVF',
    '2 ESO_Matemáticas_1_EVALUACION': '2ESO_M_1EV',
    '2 ESO_Matemáticas_2_EVALUACION': '2ESO_M_2EV',
    '2 ESO_Matemáticas_EVALUACION_FINAL': '2ESO_M_EVF',
    '2 ESO_Media curso_1_EVALUACION': '2ESO_MEDIA_1EV',
    '2 ESO_Media curso_2_EVALUACION': '2ESO_MEDIA_2EV',
    '2 ESO_Media curso_EVALUACION_FINAL': '2ESO_MEDIA_EVF',
    # CAMBIOS PARA 1FPB
    '1 FPB_Castellano_1_EVALUACION': '1FPB_C_1EV',
    '1 FPB_Castellano_2_EVALUACION': '1FPB_C_2EV',
    '1 FPB_Castellano_EVALUACION_FINAL': '1FPB_C_EVF',
    '1 FPB_E. Física_1_EVALUACION': '1FPB_EF_1EV',
    '1 FPB_E. Física_2_EVALUACION': '1FPB_EF_2EV',
    '1 FPB_E. Física_EVALUACION_FINAL': '1FPB_EF_EVF',
    '1 FPB_Inglés_1_EVALUACION': '1FPB_I_1EV',
    '1 FPB_Inglés_2_EVALUACION': '1FPB_I_2EV',
    '1 FPB_Inglés_EVALUACION_FINAL': '1FPB_I_EVF',
    '1 FPB_Matemáticas_1_EVALUACION': '1FPB_M_1EV',
    '1 FPB_Matemáticas_2_EVALUACION': '1FPB_M_2EV',
    '1 FPB_Matemáticas_EVALUACION_FINAL': '1FPB_M_EVF',
    '1 FPB_Media curso_1_EVALUACION': '1FPB_MEDIA_1EV',
    '1 FPB_Media curso_2_EVALUACION': '1FPB_MEDIA_2EV',
    '1 FPB_Media curso_EVALUACION_FINAL': '1FPB_MEDIA_EVF',
    '1 FPB_Ámbito Profesional 1º_1_EVALUACION': '1FPB_P_1EV',
    '1 FPB_Ámbito Profesional 1º_2_EVALUACION': '1FPB_P_2EV',
    '1 FPB_Ámbito Profesional 1º_EVALUACION_FINAL': '1FPB_P_EVF',
    '1 FPB_Ámbito de Ciencias Aplicadas 1º_1_EVALUACION': '1FPB_CA_1EV',
    '1 FPB_Ámbito de Ciencias Aplicadas 1º_2_EVALUACION': '1FPB_CA_2EV',
    '1 FPB_Ámbito de Ciencias Aplicadas 1º_EVALUACION_FINAL': '1FPB_CA_EVF',
    '1 FPB_Ámbito de Comunicación y Ciencias Sociales 1º_1_EVALUACION': '1FPB_CS_1EV',
    '1 FPB_Ámbito de Comunicación y Ciencias Sociales 1º_2_EVALUACION': '1FPB_CS_2EV',
    '1 FPB_Ámbito de Comunicación y Ciencias Sociales 1º_EVALUACION_FINAL': '1FPB_CS_EVF',
    # ID_ESTUDIANTE A ID
    'ID_ESTUDIANTE': 'ID',
    # CAMBIOS PARA CURSOS REPETIDOS
    'REPITIENDO_1_ESO': 'REP_1ESO',
    'REPITIENDO_2_ESO': 'REP_2ESO',
    'REPITIENDO_1_FPB': 'REP_1FPB',
    # CAMBIOS PARA EL TOTAL DE RECUPERACIONES A LAS QUE SE HA PRESENTADO
    'RECUPERACIONES_1_RECUPERACION': 'REC_1EV',
    'RECUPERACIONES_2_RECUPERACION': 'REC_2EV',
    'RECUPERACIONES_EV_EXTRAORDINARIA': 'REC_EVEX',
    # CAMBIOS PARA PROMOCIONA (CLASE FINAL - PROMOCIONA DE 1FPB A 2FPB O NO)
    'PROMOCIONA_1_FPB': 'PROMOCIONA'
}

result.rename(columns=nuevos_nombres, inplace=True)

result.head()

,ID,1ESO_C_1EV,1ESO_C_2EV,1ESO_C_EVF,1ESO_EF_1EV,1ESO_EF_2EV,1ESO_EF_EVF,1ESO_I_1EV,1ESO_I_2EV,1ESO_I_EVF,...,1FPB_CS_1EV,1FPB_CS_2EV,1FPB_CS_EVF,REP_1ESO,REP_2ESO,REP_1FPB,REC_1EV,REC_2EV,REC_EVEX,PROMOCIONA
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,5.0,5.0,0,0,0,2.0,0.0,1.0,N
1,2,6.0,5.0,5.0,5.0,5.0,8.0,3.0,4.0,3.0,...,5.0,5.0,5.0,0,0,1,4.0,4.0,2.0,N
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,5.0,6.0,0,0,0,0.0,0.0,1.0,S
4,4,2.0,3.0,4.0,3.0,5.0,4.0,5.0,5.0,5.0,...,5.0,5.0,6.0,1,0,0,2.0,0.0,2.0,S
5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,7.0,8.0,0,0,0,0.0,0.0,0.0,S


In [20]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37 entries, 0 to 44
Data columns (total 50 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              37 non-null     int64  
 1   1ESO_C_1EV      27 non-null     float64
 2   1ESO_C_2EV      27 non-null     float64
 3   1ESO_C_EVF      27 non-null     float64
 4   1ESO_EF_1EV     27 non-null     float64
 5   1ESO_EF_2EV     27 non-null     float64
 6   1ESO_EF_EVF     27 non-null     float64
 7   1ESO_I_1EV      27 non-null     float64
 8   1ESO_I_2EV      27 non-null     float64
 9   1ESO_I_EVF      27 non-null     float64
 10  1ESO_M_1EV      27 non-null     float64
 11  1ESO_M_2EV      27 non-null     float64
 12  1ESO_M_EVF      27 non-null     float64
 13  1ESO_MEDIA_1EV  27 non-null     float64
 14  1ESO_MEDIA_2EV  27 non-null     float64
 15  1ESO_MEDIA_EVF  27 non-null     float64
 16  2ESO_C_1EV      28 non-null     float64
 17  2ESO_C_2EV      28 non-null     float

# Guardado del nuevo dataset transformado en csv como 'DATA_HIST_FINAL.csv'

In [21]:
# Guarda el DataFrame en un archivo CSV
archivo_csv = 'DATA_HIST_FINAL.csv'
result.to_csv(archivo_csv, index=False)